In [ ]:
from normalization.normalize import Normalize
from sklearn.datasets import fetch_openml

data = fetch_openml(name='credit-g')
denormalized_table = data.data.join(data.target)  # join on indexes
list(denormalized_table.columns.values)
norm = Normalize(denormalized_table=denormalized_table, denormalized_table_name="target")
norm.create_entity(source_entity_name="target", target_entity_name="checking_status_type", columns=['checking_status'],
                   distinct_values=True)
norm.create_entity(source_entity_name="target", target_entity_name="savings_status_type", columns=['savings_status'],
                   distinct_values=True)
credit_cols = ['duration', 'purpose', 'installment_commitment', 'credit_history']
norm.create_entity(source_entity_name='target', target_entity_name='credit', columns=credit_cols, distinct_values=True)
norm.entity_set.keys()

person_cols = ['employment', 'personal_status', 'other_parties', 'age', 'other_payment_plans', 'existing_credits',
               'job', 'num_dependents', 'own_telephone', 'foreign_worker']
norm.create_entity(source_entity_name='target', target_entity_name='person', columns=person_cols, distinct_values=True)
norm.create_entity(source_entity_name='person', target_entity_name='personal_status_type', columns=['personal_status'],
                   distinct_values=True)
norm.create_entity(source_entity_name='person', target_entity_name='employment_type', columns=['employment'],
                   distinct_values=True)
norm.create_entity(source_entity_name='person', target_entity_name='job_type', columns=['job'], distinct_values=True)
property_cols = ['residence_since', 'property_magnitude', 'housing']
norm.create_entity(source_entity_name='target', target_entity_name='property', columns=property_cols,
                   distinct_values=True)

norm.primary_keys
norm.relations
norm.entity_set.keys()
for entity in norm.entity_set.keys():
    print(
        f"ALTER TABLE {entity} ADD CONSTRAINT PK_{entity} PRIMARY KEY ({norm.primary_keys[entity]});")

for parent_table in norm.relations:
    for child_table in norm.relations[parent_table]:
        print(f"ALTER TABLE {child_table} ADD CONSTRAINT FK_{child_table}_{parent_table} "
              f"FOREIGN KEY ({norm.relations[parent_table][child_table][1]}) "
              f"REFERENCES {parent_table}({norm.relations[parent_table][child_table][0]}));")
norm.persist_entity_set(database_name="credit_exp3")

